In [ ]:
# Initial Python environment setup...
import numpy as np # linear algebra
import pandas as pd # CSV file I/O (e.g. pd.read_csv)
import os # reading the input files we have access to

print(os.listdir('../input'))

In [ ]:
train_data =  pd.read_csv('../input/train.csv', nrows = 10_000_000)
print(train_data.dtypes)
test_data = pd.read_csv('../input/test.csv')

In [ ]:
# Given a dataframe, add two new features 'abs_diff_longitude' and
# 'abs_diff_latitude' reprensenting the "Manhattan vector" from
# the pickup location to the dropoff location.
def add_travel_vector_features(df):
    df['Difference_longitude'] = (df.dropoff_longitude - df.pickup_longitude).abs()
    df['Difference_latitude'] = (df.dropoff_latitude - df.pickup_latitude).abs()

add_travel_vector_features(train_data)
add_travel_vector_features(test_data)

In [ ]:
print(train_data.isnull().sum())

In [ ]:
print('Old size: %d' % len(train_data))
train_data = train_data.dropna(how = 'any', axis = 'rows')
print('New size: %d' % len(train_data))

In [ ]:
plot = train_data.iloc[:2000].plot.scatter('Difference_longitude', 'Difference_latitude')

In [ ]:
print('Old size: %d' % len(train_data))
train_data = train_data[(train_data.Difference_longitude < 5.0) & (train_data.Difference_latitude < 5.0)]
print('New size: %d' % len(train_data))

In [ ]:
ls1 = list(train_data['pickup_datetime'])
for i in range(len(ls1)):
    ls1[i] = ls1[i][11:-7:]
train_data['pickuptime'] = ls1

ls1 = list(test_data['pickup_datetime'])
for i in range(len(ls1)):
    ls1[i] = ls1[i][11:-7:]
test_data['pickuptime'] = ls1


In [ ]:
train_data.head()

In [ ]:
ls1 = list(train_data['pickup_datetime'])
for i in range(len(ls1)):
    ls1[i] = ls1[i][:-4:]
    ls1[i] = pd.Timestamp(ls1[i])
    ls1[i] = ls1[i].weekday()
train_data['Weekday'] = ls1

ls1 = list(test_data['pickup_datetime'])
for i in range(len(ls1)):
    ls1[i] = ls1[i][:-4:]
    ls1[i] = pd.Timestamp(ls1[i])
    ls1[i] = ls1[i].weekday()
test_data['Weekday'] = ls1

In [ ]:
test_data.head()

In [ ]:
train_data.drop('pickup_datetime', inplace=True, axis=1)
test_data.drop('pickup_datetime', inplace=True, axis=1)

In [ ]:
train_data['Weekday'].replace(to_replace=[i for i in range(0,7)],
                              value=['Monday', 'Tuesday','Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday' ],
                              inplace=True)
test_data['Weekday'].replace(to_replace=[i for i in range(0,7)],
                              value=['Monday', 'Tuesday','Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday' ],
                              inplace=True)

In [ ]:
train_hot = pd.get_dummies(train_data['Weekday'])
test_hot = pd.get_dummies(train_data['Weekday'])
train_data = pd.concat([train_data, train_hot], axis=1)
test_data = pd.concat([test_data, test_hot], axis=1)

In [ ]:
train_data.drop('Weekday', inplace=True, axis=1)
test_data.drop('Weekday', inplace=True, axis=1)

In [ ]:

test_data['pickuptime'] = test_data['pickuptime'].astype(str)

In [ ]:
test_data.tail()


In [ ]:
ls1 = list(train_data['pickuptime'])
for i in range(len(ls1)):
    z=ls1[i].split(':')
    ls1[i]=int(z[0])*100+int(z[1])
train_data['pickuptime'] = ls1


test_data = test_data.dropna()
ls1 = list(test_data['pickuptime'])
for i in range(len(ls1)):
    z=ls1[i].split(':')
    ls1[i]=int(float(z[0]))*100+int(float(z[1]))
test_data['pickuptime'] = ls1    

In [ ]:
train_data.head()

In [ ]:
R = 6373.0
lat1 = np.asarray(np.radians(train_data['pickup_latitude']))
lon1 = np.asarray(np.radians(train_data['pickup_longitude']))
lat2 = np.asarray(np.radians(train_data['dropoff_latitude']))
lon2 = np.asarray(np.radians(train_data['dropoff_longitude']))

dlon = lon2 - lon1
dlat = lat2 - lat1
ls1 = []

a = np.sin(dlat/2)**2 + np.cos(lat2) * np.sin(dlon /2)**2
c = 2*np.arctan2(np.sqrt(a), np.sqrt(1-a))
distance = R*c

train_data['Distance'] = np.asarray(distance)*0.621

lat1 = np.asarray(np.radians(test_data['pickup_latitude']))
lon1 = np.asarray(np.radians(test_data['pickup_longitude']))
lat2 = np.asarray(np.radians(test_data['dropoff_latitude']))
lon2 = np.asarray(np.radians(test_data['dropoff_longitude']))

dlon = lon2 - lon1
dlat = lat2 - lat1

a = np.sin(dlat/2)**2 + np.cos(lat2) * np.sin(dlon /2)**2
c = 2*np.arctan2(np.sqrt(a), np.sqrt(1-a))
distance = R*c

test_data['Distance'] = np.asarray(distance)*0.621


In [ ]:
R = 6373.0
lat1 = np.asarray(np.radians(train_data['pickup_latitude']))
lon1 = np.asarray(np.radians(train_data['pickup_longitude']))
lat2 = np.asarray(np.radians(train_data['dropoff_latitude']))
lon2 = np.asarray(np.radians(train_data['dropoff_longitude']))

lat3 = np.zeros(len(train_data)) + np.radians(40.641311)
lon3 = np.zeros(len(train_data)) + np.radians(-73.7781391)
dlon_pickup = lon3 - lon1
dlat_pickup = lat3 - lat1
dlon_dropoff = lon3 - lon2
dlat_dropoff = lat3 - lat2

a1 = np.sin(dlat_pickup/2)**2 + np.cos(lat1)*np.cos(lat3) * np.sin(dlon_pickup /2)**2
c1 = 2*np.arctan2(np.sqrt(a1), np.sqrt(1-a1))
distance1 = R*c1

train_data['Pickup_Distance_airport'] = np.asarray(distance1)*0.621
a2 = np.sin(dlat_dropoff/2)**2 + np.cos(lat2)*np.cos(lat3) * np.sin(dlon_dropoff /2)**2
c2 = 2*np.arctan2(np.sqrt(a2), np.sqrt(1-a2))
distance2 = R*c2

train_data['Dropoff_Distance_airport'] = np.asarray(distance2)*0.621

lat1 = np.asarray(np.radians(test_data['pickup_latitude']))
lon1 = np.asarray(np.radians(test_data['pickup_longitude']))
lat2 = np.asarray(np.radians(test_data['dropoff_latitude']))
lon2 = np.asarray(np.radians(test_data['dropoff_longitude']))

lat3 = np.zeros(len(test_data)) + np.radians(40.641311)
lon3 = np.zeros(len(test_data)) + np.radians(-73.7781391)
dlon_pickup = lon3 - lon1
dlat_pickup = lat3 - lat1
dlon_dropoff = lon3 - lon2
dlat_dropoff = lat3 - lat2

a1 = np.sin(dlat_pickup/2)**2 + np.cos(lat1)*np.cos(lat3) * np.sin(dlon_pickup /2)**2
c1 = 2*np.arctan2(np.sqrt(a1), np.sqrt(1-a1))
distance1 = R*c1

test_data['Pickup_Distance_airport'] = np.asarray(distance1)*0.621

a2 = np.sin(dlat_dropoff/2)**2 + np.cos(lat2)*np.cos(lat3) * np.sin(dlon_dropoff /2)**2
c2 = 2*np.arctan2(np.sqrt(a2), np.sqrt(1-a2))
distance2 = R*c2

test_data['Dropoff_Distance_airport'] = np.asarray(distance2)*0.621

In [ ]:
test_data.head()

In [ ]:
train_data['Distance'] = np.round(train_data['Distance'], 2)
train_data['Pickup_Distance_airport'] = np.round(train_data['Pickup_Distance_airport'],2)
train_data['Dropoff_Distance_airport'] = np.round(train_data['Dropoff_Distance_airport'],2)

test_data['Distance'] = np.round(test_data['Distance'], 2)
test_data['Pickup_Distance_airport'] = np.round(test_data['Pickup_Distance_airport'],2)
test_data['Dropoff_Distance_airport'] = np.round(test_data['Dropoff_Distance_airport'],2)

In [ ]:
test_data.head()

In [ ]:
# train_data.drop(['pickup_longitude', 'pickup_latitude', 'dropoff_latitude', 'dropoff_longitude'], axis=1, inplace=True)
# test_data.drop(['pickup_longitude', 'pickup_latitude', 'dropoff_latitude', 'dropoff_longitude'], axis=1, inplace=True)
train_data.drop(['pickup_longitude'], axis=1, inplace=True)
test_data.drop(['pickup_longitude'], axis=1, inplace=True)

In [ ]:
train_data.head()

In [ ]:
train_data['Difference_longitude']=np.abs(train_data['Difference_longitude'] - np.mean(train_data['Difference_longitude']))
train_data['Difference_longitude']=train_data['Difference_longitude']/np.var(train_data['Difference_longitude'])

In [ ]:
train_data['Difference_latitude']=np.abs(train_data['Difference_latitude'] - np.mean(train_data['Difference_latitude']))
train_data['Difference_latitude']=train_data['Difference_latitude']/np.var(train_data['Difference_latitude'])

In [ ]:
test_data['Difference_longitude']=np.abs(test_data['Difference_longitude'] - np.mean(test_data['Difference_longitude']))
test_data['Difference_longitude']=test_data['Difference_longitude']/np.var(test_data['Difference_longitude'])
test_data['Difference_latitude']=np.abs(test_data['Difference_latitude'] - np.mean(test_data['Difference_latitude']))
test_data['Difference_latitude']=test_data['Difference_latitude']/np.var(test_data['Difference_latitude'])

In [ ]:
train_data.shape

In [ ]:
test_data.shape

In [ ]:
from sklearn.model_selection import train_test_split
X = train_data.drop(['key', 'fare_amount'], axis=1)
y = train_data['fare_amount']
X_train,X_test,y_train, y_test = train_test_split(X,y,test_size=0.01, random_state=80)

In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression(normalize=True)
lr.fit(X_train, y_train)
print(lr.score(X_test, y_test))

In [ ]:
pred=np.round(lr.predict(test_data.drop('key', axis=1)),2)

In [ ]:
sub = pd.DataFrame(data=pred, columns=['fare_amount'])
sub['key'] = test_data['key']
sub=sub[['key','fare_amount']]

In [ ]:
sub.set_index('key', inplace=True)

In [ ]:
sub.to_csv('Submission.csv')